<a href="https://colab.research.google.com/github/gareytwin1/alpaca-finance-lab/blob/main/stocks/trading_bot_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install alpaca-trade-api
!pip install plotly
!pip install mplfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.2 MB/s eta 0:00:00


In [5]:
!pip install alpaca-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.7/121.7 kB 5.8 MB/s eta 0:00:00


In [7]:
import alpaca_trade_api as tradeapi
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
import config
import csv
from datetime import datetime, timedelta
from google.colab import userdata
import json
import logging
import math
import mplfinance as mpf
import matplotlib.pyplot as plt
import os
import pandas as pd
import polars as pl
import plotly.graph_objects as go
import seaborn as sns
import time

In [8]:
POSITION_FILE: str = 'position.json'
SYMBOL: str = 'SPY'
MAX_TRADES_PER_DAY: int = 10        # Maximum trades allowed per day
DAILY_LOSS_LIMIT: int  = 100       # Maximum daily loss allowed (in dollars)
POSITION_SIZE_PCT: float = 0.1        # 10% of buying power per trade
COOLDOWN: int = 60 * 5              # 5 minutes cooldown in seconds
TRAILING_STOP_PCT: float = 0.02       # 2% trailing stop

# Initial position dictionary
initial_position_data = {
    "side": "long",
    "entry_price": 180.50,
    "qty": 10,
    "high": 180.50,
    "low": 180.00,
    "entry_time": datetime.now().isoformat(),
    "last_exit": ""
}

# Remove existing handlers (important in Jupyter)
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# Now set up logging to file
logging.basicConfig(
    filename='app.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filemode='a'
)

# Test it
logging.info("Logging from Jupyter is working.")

In [9]:
def create_position_file_if_not_exists() -> None:
    """Creates the position.json file with initial data if it doesn't exist."""
    if not os.path.exists(POSITION_FILE):
        try:
            with open(POSITION_FILE, 'w') as f:
                json.dump(initial_position_data, f, indent=4)
            logging.info(f"'{POSITION_FILE}' not found. Created and initialized it.")
        except Exception as e:
            logging.error(f"Error creating '{POSITION_FILE}': {e}", exc_info=True)
    else:
        logging.info(f"'{POSITION_FILE}' already exists.")

In [10]:
create_position_file_if_not_exists()

In [11]:
def load_position() -> list[dict] | None:
    """Loads the trading position from the position.json file, ensuring recovery from corruption or absence."""
    create_position_file_if_not_exists()

    try:
        with open(POSITION_FILE, 'r') as f:
            position_data: list[dict] = json.load(f)
            logging.info(f"Successfully loaded position from {POSITION_FILE}")
            return position_data

    except json.JSONDecodeError:
        logging.warning(f"Error decoding JSON from {POSITION_FILE}. File might be corrupted. Attempting to fix.")
        try:
            os.remove(POSITION_FILE)
            logging.info(f"Deleted corrupted {POSITION_FILE}. Re-creating it.")
            create_position_file_if_not_exists()
            with open(POSITION_FILE, 'r') as f:
                position_data: list[dict] = json.load(f)
                logging.info(f"Successfully recovered and reloaded {POSITION_FILE}")
                return position_data
        except Exception as e:
            logging.error(f"Failed to reload after recreating {POSITION_FILE}: {e}", exc_info=True)
            return None

    except Exception as e:
        logging.error(f"Unexpected error reading {POSITION_FILE}: {e}", exc_info=True)
        return None


In [12]:
load_position()

{'side': 'long',
 'entry_price': 180.5,
 'qty': 10,
 'high': 180.5,
 'low': 180.0,
 'entry_time': '2025-06-14T22:42:34.302472',
 'last_exit': ''}

In [13]:
def reset_position() -> None:
    """Resets the position by deleting the position.json file."""
    try:
        if os.path.exists(POSITION_FILE):
            os.remove(POSITION_FILE)
            logging.info(f"Successfully deleted {POSITION_FILE}. Position reset.")
        else:
            logging.info(f"No position file found to delete. Nothing to reset.")
    except Exception as e:
        logging.error(f"Error while trying to reset position: {e}", exc_info=True)

In [ ]:
def close_position():
    try:
        # api.close_position(SYMBOL)
        print("Position closed.")
    except Exception as e:
        print(f"Close error: {e}")

In [ ]:
def save_position(data):
    POSITION_FILE = 'position.json'
    with open(POSITION_FILE, 'w') as f:
      json.dump(data,f)
    print("Positioned save")

In [ ]:
def place_order(side, qty, price):
    try:
        # api.submit_order(
        #     symbol=SYMBOL,
        #     qty=qty,
        #     side=side,
        #     type='limit',
        #     limit_price=round(price, 2),
        #     time_in_force='gtc'
        # )
        print(f"{side.upper()} order placed at {price}")
    except Exception as e:
        print(f"Order error: {e}")

In [ ]:
def log_trade(entry_time, exit_time, side, entry_price, exit_price, pnl):
    # exists = os.path.exists(TRADE_LOG_FILE)
    # with open(TRADE_LOG_FILE, 'a') as f:
    #     if not exists:
    #         f.write("entry_time,exit_time,side,entry_price,exit_price,pnl\n")
    #     f.write(f"{entry_time},{exit_time},{side},{entry_price},{exit_price},{pnl}\n")
    print("Logged trade")

In [ ]:
def check_daily_limits():
    #List comphrension to a list of daily_list
    daily_trades = [activity for activity in activities if datetime.fromisoformat(activity["transaction_time"].isoformat()).date() == datetime.now().date()]
    print(f"daily trades: {daily_trades}")
    # Code into long way: for t in daily_trades if t.side in ['buy', 'sell']]

    filtered_trades = []
    for t in daily_trades:
        if t['side'] in ['buy', 'sell']:
            filtered_trades.append(t)
    # Lets sum all pnl for today:
    today_pnl = sum(trade["realized_pl"] for trade in filtered_trades)
    print(f"Today's total PNL: {today_pnl}")

    try:
        trade_count = len([t for t in daily_trades if t['side'] in ['buy', 'sell']])
        print(f"Trade count: {trade_count}")
        today_pnl = sum(trade["realized_pl"] for trade in filtered_trades)
        print(f"Today's total PNL: {today_pnl}")
        print(f"returning: {trade_count, today_pnl}")
        return trade_count, today_pnl
    except Exception as e:
        print(f"Runtime error {e}")
        return 0, 0

In [ ]:
def get_data(symbol, timeframe='1m', limit=30):
    try:
        ticker = yf.Ticker(SYMBOL)

        if timeframe == '1m':
            df = ticker.history(period="1d", interval="1m")
        elif timeframe == '5m':
            df = ticker.history(period="1d", interval="5m")

        df = ticker.history(period="1d", interval="1m")
    # try:
        # api_key = userdata.get('ALPACA_API_KEY')
        # secret_key = userdata.get('ALPACA_SECRET_KEY')
        # base_url = userdata.get('ALPACA_API_BASE_URL') # Default to paper trading URL

        # if not api_key or not secret_key:
        #     print("Alpaca API keys not found. Please set APCA_API_KEY_ID and APCA_API_SECRET_KEY environment variables.")
        # else:
        #     api = tradeapi.REST(api_key, secret_key, base_url, api_version='v2')
        #     client = StockHistoricalDataClient(API_KEY, API_SECRET)

        #     end = datetime.now()
        #     start = end - timedelta(minutes=limit)

        #     request_params = StockBarsRequest(
        #         symbol_or_symbols=symbol,
        #         timeframe=TimeFrame(timeframe, TimeFrameUnit.Minute),
        #         start=start,
        #         end=end
        #     )

        #     bars = client.get_stock_bars(request_params).df
        #     df = bars.reset_index().tail(limit)
        #     df.set_index('timestamp', inplace=True)
        return df
    except Exception as e:
        print(f"Data error: {e}")
        return pd.DataFrame()

In [ ]:
# Interactive chart with plotly
def create_candlestick_chart(df):
    # Create candlestick trace
    candlestick = go.Candlestick(
        x=df.index,
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close']
    )

    # Create figure and add trace
    fig = go.Figure(data=[candlestick])

    # Update layout
    fig.update_layout(
        title=f'{SYMBOL} Interactive Candlestick Chart',
        xaxis_title='Time',
        yaxis_title='Price',
        xaxis_rangeslider_visible=False # Hide range slider
    )

    # Show the plot
    fig.show()

In [ ]:
def calculate_ema(data, period=9):
    return data['Close'].ewm(span=period, adjust=False).mean()

In [ ]:
def calculate_rsi(data, period=14):
    delta = data['Close'].diff()
    gain = delta.where(delta > 0, 0).rolling(window=period).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

In [ ]:
def average_volume(data, window=20):
    return data['Volume'].rolling(window=window).mean()

In [ ]:
def calculate_slope(series, window=5):
    return series.diff().rolling(window=window).mean()

In [ ]:
def check_missing_values(df):
    missing_values = df.isnull().sum()
    missing_values = missing_values[missing_values > 0]
    return missing_values

In [ ]:
# Remove columns 'Dividends' and 'Stock Splits'
def remove_dividends_splits(df, columns):
    try:
        df = df.drop(columns=columns)
    except Exception as e:
        print(f"Data error: {e}")
    return df

In [ ]:
def trade():

    position = load_position()
    print(f"position: {position}")

    # --- Modified Cooldown Check ---
    now = datetime.now()
    print(f"now: {now}")

    # Check if last_exit is a non-empty string before attempting to parse
    last_exit_str = position.get('last_exit', '') # Use .get to safely access, default to ''
    print(f"position['last_exit'] string: '{last_exit_str}'")

    if last_exit_str: # Check if the string is not empty
        try:
            last_exit = datetime.fromisoformat(last_exit_str)
            print(f"last_exit (parsed): {last_exit}")

            # Cooldown logic only applies if last_exit was successfully parsed
            formatted_time = last_exit.strftime("%H:%M:%S")
            print(f"formatted_time: {formatted_time}")
            time_from_last_exit = int((now - last_exit).total_seconds())
            print(f"time_from_last_exit: {time_from_last_exit}")
            time_left = COOLDOWN - time_from_last_exit
            print(f"time_left: {time_left}")

            if (time_left < COOLDOWN) and (time_left > 0):
                print(f"Cooldown active until {time_left} seconds from now. Last trade was at {formatted_time}")
                return # Exit the trade function if cooldown is active
        except ValueError:
            # Handle cases where the string is not a valid isoformat
            print(f"Warning: 'last_exit' value '{last_exit_str}' is not a valid ISO format. Assuming no recent exit for cooldown.")
            # Treat as if there was no recent exit (cooldown not active)
            time_left = COOLDOWN + 1 # Ensure time_left is greater than COOLDOWN
    else:
        # If last_exit_str is empty, assume no recent exit (cooldown not active)
        print("'last_exit' is empty. Assuming no recent exit for cooldown.")
        time_left = COOLDOWN + 1 # Ensure time_left is greater than COOLDOWN

    # Rest of your trade logic continues here if cooldown is not active
    trade_count, daily_pnl = check_daily_limits()
    if trade_count >= MAX_TRADES_PER_DAY or daily_pnl <= -DAILY_LOSS_LIMIT:
        print("Daily limit reached.")
        return

    df_1m = get_data(SYMBOL, '1m', 30)
    df_5m = get_data(SYMBOL, '5m', 30)

    if df_1m.empty or df_5m.empty:
        print("Data unavailable.")
        return

    df_1m['ema9'] = calculate_ema(df_1m)
    df_1m['rsi'] = calculate_rsi(df_1m)
    df_1m['avg_volume'] = average_volume(df_1m)

    df_5m['ema50'] = calculate_ema(df_5m, 50)
    df_5m['slope50'] = calculate_slope(df_5m['ema50'])

    last = df_1m.iloc[-1].copy()
    #print("Raw last row values:")
    for k, v in last.items():
        print(f"  {k}: {v}")

    # convert the values in the last row of that DataFrame to a consistent numeric format.
    numerical_columns = df_1m.select_dtypes(include='number').columns.tolist()
    #print(f"Numeric columns: {numerical_columns}")
    for col in numerical_columns:
        try:
            val = last[col]
            if pd.isna(val) or str(val).strip() in ['', 'None', 'nan', 'NaN']:
                raise ValueError(f"Missing or empty value for {col}")
                val = float(val) if isinstance(val, (int, float)) else float(str(val).replace(',', '').strip())
                last[col] = val
        except Exception as e:
            print(f"Data conversion error for '{col}': {e}")

    #last

    slope = float(df_5m['slope50'].iloc[-1])
    print(f"Slope: {slope}")
    side = position.get('side')
    print(f"Side: {side}")
    qty = position.get('qty', 0)
    print(f"Qty: {qty}")
    entry_price = position.get('entry_price', 0)
    print(f"Entry price: {entry_price}")
    buying_power = 100000 #float(api.get_account().cash)
    trade_qty = int((buying_power * POSITION_SIZE_PCT) / last['Close'])
    print(f"Trade qty: {trade_qty}")

    # No position
    if not side:
        print("No position found")
        save_position({'side': 'long', 'entry_price': 100, 'qty': trade_qty, 'high': 200, 'entry_time': now.isoformat()})
        # if last['Close'] > last['Open'] and last['Close'] > last['ema9'] and last['rsi'] < 65 and last['Volume'] > last['avg_volume'] and slope > 0:
        #     price = last['Close'] * 1.001
        #     place_order('buy', trade_qty, price)
        #     save_position({'side': 'long', 'entry_price': price, 'qty': trade_qty, 'high': price, 'entry_time': now.isoformat()})

        # elif last['Close'] < last['Open'] and last['Close'] < last['ema9'] and last['rsi'] > 55 and last['Volume'] > last['avg_volume'] and slope < 0:
        #     price = last['Close'] * 0.999
        #     place_order('sell', trade_qty, price)
        #     save_position({'side': 'short', 'entry_price': price, 'qty': trade_qty, 'low': price, 'entry_time': now.isoformat()})

    # Long Position
    elif side == 'long':
        print("Long Position")
        position['high'] = max(position['high'], last['Close'])
        trail_stop = position['high'] * (1 - TRAILING_STOP_PCT)
        print(f"[LONG] Comparing last['Close'] = {last['Close']} (type: {type(last['Close'])}) to trail_stop = {trail_stop}")
        last_close = float(last['Close'])
        if last_close <= trail_stop or last_close < last['ema9']:
            close_position()
            exit_price = last['Close']
            pnl = (exit_price - entry_price) * qty
            log_trade(position['entry_time'], now.isoformat(), side, entry_price, exit_price, pnl)
            #reset_position()
            save_position({'last_exit': now.isoformat()})

    # Short Position
    elif side == 'short':
        print("Short Position")
        position['low'] = min(position['low'], last['Close'])
        trail_stop = position['low'] * (1 + TRAILING_STOP_PCT)
        print(f"[SHORT] Comparing last['Close'] = {last['Close']} (type: {type(last['Close'])}) to trail_stop = {trail_stop}")
        last_close = float(last['Close'])
        if last_close >= trail_stop or last_close > last['ema9']:
            close_position()
            exit_price = last['Close']
            pnl = (entry_price - exit_price) * qty
            log_trade(position['entry_time'], now.isoformat(), side, entry_price, exit_price, pnl)
            #reset_position()
            save_position({'last_exit': now.isoformat()})


In [ ]:
try:
    trade()
except Exception as e:
    print(f"Runtime error: {e}")
#time.sleep(60)

'position.json' already exists.
Successfully loaded position from position.json
position: {'last_exit': '2025-06-08T20:27:25.835136'}
now: 2025-06-08 20:27:46.097993
position['last_exit'] string: '2025-06-08T20:27:25.835136'
last_exit (parsed): 2025-06-08 20:27:25.835136
formatted_time: 20:27:25
time_from_last_exit: 20
time_left: 0
daily trades: [{'symbol': 'TSLA', 'side': 'buy', 'transaction_time': datetime.datetime(2025, 6, 8, 18, 23, 31, 29636), 'realized_pl': 0.0}, {'symbol': 'TSLA', 'side': 'sell', 'transaction_time': datetime.datetime(2025, 6, 8, 18, 53, 31, 29649), 'realized_pl': 15.75}, {'symbol': 'TSLA', 'side': 'buy', 'transaction_time': datetime.datetime(2025, 6, 8, 19, 38, 31, 29653), 'realized_pl': 0.0}, {'symbol': 'TSLA', 'side': 'sell', 'transaction_time': datetime.datetime(2025, 6, 8, 19, 53, 31, 29657), 'realized_pl': -5.25}]
Today's total PNL: 10.5
Trade count: 4
Today's total PNL: 10.5
returning: (4, 10.5)
  Open: 295.6700134277344
  High: 295.69000244140625
  Low: 2